In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/cs6471proj2/amazon

/content/drive/MyDrive/cs6471proj2/amazon


In [ ]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = '../corenlp'
# stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

# Examine the CoreNLP installation folder to make sure the installation is successful
!ls $CORENLP_HOME


     |████████████████████████████████| 286kB 12.4MB/s 
build.xml				  jollyday.jar
corenlp.sh				  LIBRARY-LICENSES
CoreNLP-to-HTML.xsl			  LICENSE.txt
ejml-core-0.39.jar			  Makefile
ejml-core-0.39-sources.jar		  patterns
ejml-ddense-0.39.jar			  pom-java-11.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  SemgrexDemo.java
input.txt.xml				  ShiftReduceDemo.java
javax.activation-api-1.2.0.jar		  slf4j-api.jar
javax.activation-api-1.2.0-sources.jar	  slf4j-simple.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.2.0.jar
javax.json.jar				  stanford-corenlp-4.2.0-javadoc.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.2.0-models.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   stanford-corenlp-4.2.0-sources.jar
jaxb-core-2.3.0.1.jar			  StanfordCoreNlpDemo.java
jaxb-core-2.3.0.1-sources.jar		  StanfordDependenciesManual.pdf
jaxb-impl-2.4.0-b18083

In [ ]:
client.stop()

In [ ]:
import re
import time
import glob
import json
from stanza.server import CoreNLPClient
import os


def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

'''
The idea here is to build location set, country set, province set, nationality set
Also, the dictionary for stats analysis
Based on the reviews that have helpfulness scores.
'''

all_file_glob = glob.glob('*.json')
# print(all_file)

# all_file = ['reviews_Musical_Instruments_5.json', 'reviews_Office_Products_5.json', 'reviews_Amazon_Instant_Video_5.json', 'reviews_Digital_Music_5.json']
# done ['reviews_Musical_Instruments_5.json', 'reviews_Office_Products_5.json', 'reviews_Cell_Phones_and_Accessories_5.json', ]
all_file = ['reviews_Toys_and_Games_5.json']

# check file exits:
for file in all_file:
    print(file in all_file_glob)

location_titles = set(["COUNTRY", 'CITY', 'STATE_OR_PROVINCE', 'LOCATION', 'NATIONALITY'])
for name in all_file:
    print(name)

    # clean the ram for every file
    # Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
    client = CoreNLPClient(
        # annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], 
        annotators=['tokenize','ssplit', 'ner'], 
        memory='23G', 
        endpoint='http://localhost:9001',
        be_quiet=True)
    print(client)
    time.sleep(10)

    # Start the background server and wait for some time
    # Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
    client.start()

    country_count = 0
    state_count = 0
    city_count = 0
    county_count = 0
    year_stats = {str(i):0 for i in range(1995, 2015, 1)}
    country_stats = {}
    loc_stats = {}
    province_stats = {}
    nationaily_stats = {}
    country_set = set()
    loc_set = set()
    province_set = set()
    nationaily_set = set()

    # things to save
    stats = {'COUNTRY':{}, 'CITY':{}, 'STATE_OR_PROVINCE':{}, 'LOCATION':{}, 'NATIONALITY':{},'helpfulness':[], 'year':[], 'overall':{}}
    sets = {'COUNTRY':set(), 'CITY':set(), 'STATE_OR_PROVINCE':set(), 'LOCATION':set(),  'NATIONALITY':set()}
    helpful_Review_IDs = []
    evaluated_Review_IDs = []

    folder_path = name.split('.')[0]+'_all'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    outfile = open('/'.join([folder_path, name.split('.')[0]]) + '_feature.json', 'a')
    for idx, line in enumerate(open(name, 'r')):
        if (idx+1)%5000 == 0: print(idx)

        # load line
        this_line = json.loads(line)
        evaluated_Review_IDs.append(this_line["reviewerID"])
        # if this_line['helpful'][1] < 1:  continue
        info_line = {"reviewerID": this_line["reviewerID"],'COUNTRY':[], 'CITY':[], 'STATE_OR_PROVINCE':[], 'LOCATION':[], 'NATIONALITY':[], 'helpfulness':this_line['helpful'], 'year':this_line['reviewTime'], 'rate':this_line['overall']}
        # helpful_Review_IDs.append(this_line["reviewerID"])
        stats['year'].append(this_line['reviewTime'].split(" ")[-1])
        stats['helpfulness'].append(this_line['helpful'])
        if this_line['overall'] not in stats['overall'].keys():
            stats['overall'][this_line['overall']] = 1
        else:
            stats['overall'][this_line['overall']] += 1

        # NER
        text = normalize_sentence(this_line['reviewText'])
        document = client.annotate(text)
        for sent in document.sentence:
            for m in sent.mentions:
                if m.entityType in location_titles:
                    info_line[m.entityType].append(m.entityMentionText) # m.entityMentionText, m.entityTyp = 'GA', 'STATE_OR_PROVINCE'
                    if m.entityMentionText not in sets[m.entityType]:
                        stats[m.entityType][m.entityMentionText] = [1, set([this_line["reviewerID"]])]
                        sets[m.entityType].add(m.entityMentionText)
                    else:
                        stats[m.entityType][m.entityMentionText][1].add(this_line["reviewerID"])
                        stats[m.entityType][m.entityMentionText][0] = len(stats[m.entityType][m.entityMentionText][1])


        json.dump(info_line, outfile)
        outfile.write('\n')
    
    outfile.close()

    outfile = open(folder_path+'/'+name.split('.')[0] + '_evaluated_review_ID.json', 'w')
    json.dump(evaluated_Review_IDs, outfile)
    outfile.close()

    # change set to list
    for type_ in location_titles:
        texts = stats[type_].keys()
        for text in texts:
            stats[type_][text] = [stats[type_][text][0], list(stats[type_][text][1])]
    
    outfile = open(folder_path + '/' + name.split('.')[0] + '_stats.json', 'w')
    json.dump(stats, outfile)
    outfile.close()

    # print(stats)
    print(sets)
    # print(helpful_Review_IDs)

    client.stop()


2021-04-13 02:04:52 INFO: Writing properties to tmp file: corenlp_server-43f7b8c78c5f4be9.props


True
reviews_Toys_and_Games_5.json


2021-04-13 02:05:02 INFO: Starting server with command: java -Xmx23G -cp ../corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-43f7b8c78c5f4be9.props -annotators tokenize,ssplit,ner -preload -outputFormat serialized


4999
9999
14999
19999
24999
29999
34999
39999
44999
49999
54999
59999
64999
69999
74999
79999
84999
89999
94999
99999
104999
109999
114999
119999
124999
129999
134999
139999
144999
149999
154999
159999
164999
{'COUNTRY': {'fold', 'China Egypt Peru', 'togo', 'Great Britain Canada', 'italy', 'Finland', 'China Korea', 'UK', 'Peru', 'Canada', 'Ukraine', 'Iceland', 'Austria', 'Saudi Arabia', 'India', 'UNITED STATES', 'Iraq', 'Russia India China Korea Japan', 'San Marino', 'gdr', 'hind', 'Ch', 'chad', 'PH', 'South Africa', 'Switzerland', 'Romania', 'Germany Switzerland Austria Netherlands', 'Russia Japan', 'united states', 'ENGLAND', 'the states', 'Ph', 'Germany Austria', 'PRC', 'Guatemala', 'Great Britain France Germany Austria Hungary Italy Russia', 'guinea', 'thailand', 'Netherlands', 'Uganda', 'America', 'canada', 'South Korea', 'Indonesia', 'Hungary', 'mexico', 'TURKEY', 'Vietnam', 'Sudan', 'Philippines Indonesia', 'pH', 'Thailand', 'egypt', 'USA China Germany', 'roc', 'Chad Madagascar 